# Simple MD 

adapted from [this repo](https://github.com/Markus91Koch/MDLecturesWKMBP2020/blob/main/MDlecture1/MD1-Full.ipynb)

**Goal:** write a simple coarse-grained MD Code to simulate a Lennard-Jones (LJ) fluid in the NVE ensemble. This mean we will have
- simple spherical particles that interact with each other via a LJ potential
- The time evolution of the system, i.e. the movement of these atoms, results from time integrating Newton's equations of motion by using the **Velocity-Verlet** algorithm
- NVE means system has constant particle N, constant volume V, constan energy E.

## From scratch

install packages
```
%conda env update -f ../../py310lammps.yml
```

### Define basic parameters of simulation

Define the basic parameters of the simulation, e.g. number of particles, size of the box, timestep, interaction potential (LJ potential), etc.

In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [ ]:
# simulation parameters in LJ units
N = 20              # number of particles
L = 9.0             # edge length of a cubic box
nsteps = 200000     # number of time steps
dt = 0.0005         # timestep
m = 1.0             # mass of one particle
V = L*L*L           # box volume
rho = 1.0*N/V       # density


#### The Lennard-Jones Potential:

\begin{equation}
U_\mathrm{LJ}(r) = 4 \epsilon \bigg[ \bigg(\frac{\sigma}{r}\bigg)^{12}  - \bigg(\frac{\sigma}{r}\bigg)^{6} \bigg]
\end{equation}

$r$ - distance between particle centers <br>
$\epsilon$ - interaction strength (potential depth) <br>
$\sigma$ - particle "size" (root of the potential)<br>
<br>

In simulations the LJ potential (and its negative first derivate, i.e. the force) is truncated (or cut off) at a certain distance $r$. The reasoning behind this is that the potential is nearly at zero for large enough values of $r$, anyway. When particles further apart than the cutoff distance $r_\mathrm{cut}$ are ignored in the force calculation, a lot of simulation time is saved, while the particle interactions remain realistic.
<br>
<br>

The **typical cutoff distance of the LJ potential in coarse-grained MD is $r_\mathrm{cut} = 2.5$.** In this case the particles have repulsive interactions at very short distances and attractive interactions at intermediate distances. The potential is not only truncated but also shifted along the y-axis: This is neccessary to have zero energy (and zero force) exactly at the cutoff distance: $U_\mathrm{LJ}(r_\mathrm{cut}) \stackrel{!}{=} 0$ and $F(r_\mathrm{cut}) \stackrel{!}{=} 0$. Any nonzero values $U_\mathrm{LJ}(r_\mathrm{cut}), F(r_\mathrm{cut}) \neq 0$ would be a sudden "jump" in the energy (or force) and hence an unphysical artefact.
<br>
<br>

The **shifted, truncated LJ-Potential** is obtained by subtracting the function value at the cutoff:

\begin{equation} 
U_\mathrm{LJ, cut}(r) = U_\mathrm{LJ}(r) - U_\mathrm{LJ}(r_\mathrm{cut}) 
\end{equation}

##### Typical values of the cutoff distance $r_\mathrm{cut}$ :

$r_\mathrm{cut,\,short} = \min(U_\mathrm{LJ}(r)) = \sqrt[6]{2} \approx 1.12246$ - "short" cutoff, no attractive part <br>
$r_\mathrm{cut,\, long} = 2.5$ - long cutoff, with attractive part <br>

In [3]:
%conda env update -f ../../py310lammps.yml

Solving environment: \ 
  - conda-forge/linux-64::f90wrap-0.2.9-py310h3254323_0, conda-forge/linux-64::numpy-1.24.1-py310h8deb116_0
  - conda-forge/linux-64::f90wrap-0.2.11-py310h5b92bf0_1, conda-forge/linux-64::numpy-1.23.5-py310h53a5b5fdone

mlip-1.0             | 629 KB    |                                       |   0% 
gsl-2.7              | 3.2 MB    |                                       |   0% 

clang-15-15.0.3      | 786 KB    |                                       |   0% 


f90wrap-0.2.9        | 151 KB    |                                       |   0% 



libssh2-1.10.0       | 234 KB    |                                       |   0% 




n2p2-2.1.4           | 19.1 MB   |                                       |   0% 





binutils_impl_linux- | 5.2 MB    |                                       |   0% 






pkg-config-0.29.2    | 120 KB    |                                       |   0% 







numpy-1.24.1         | 6.3 MB    |                                       |   0% 

In [ ]:
#